In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings(action='ignore', category=DeprecationWarning)
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

In [2]:
#import a bunch of stuff thats used later
import numpy as np
import pandas as pd
import sklearn
from sklearn.metrics import precision_recall_fscore_support
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier
from sklearn.ensemble.bagging import BaggingClassifier
from sklearn.ensemble.forest import ExtraTreesClassifier
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV

In [3]:


creditData = pd.read_csv('Desktop\School\Predictive Analytics w Python\Credit Risk Project\heloc_dataset_v1.csv', na_filter = True, keep_default_na = False)
copy = creditData.copy() # Copy Data Just in Case, i guess?

In [4]:
#PreProcessing Functions
from sklearn.preprocessing import OrdinalEncoder

def binarize_labels(df): #function to binarize the y-labels
    rsk_cat = df.RiskPerformance
    rsk_cat = rsk_cat.values.reshape(-1,1)
    ordinal_encoder = OrdinalEncoder()
    rsk_cat_enc = ordinal_encoder.fit_transform(rsk_cat)
    df.RiskPerformance = rsk_cat_enc
    return df
def NA_fixer(df):
    cols = df.columns #get column names

    df_with_NAs = df[df > -7] #compute values less than -6 as Nas
    df_with_NAs = df_with_NAs.dropna(how = 'all') #drop the rows that contain only missing data

    cols_to_impute = []
    for i in range(len(cols)):
        if (len(df_with_NAs[cols[i]].dropna()) / len(df_with_NAs)) > 0.9:
            cols_to_impute.append(cols[i]) # Collect columns where NAs make up 10% or less of the data

    drop_cols = set(cols) - set(cols_to_impute)    
    df = df_with_NAs.drop(drop_cols, axis = 1) # Drop Columns where NAs make up more than 10% of the data (5 of them)
    return df


In [5]:
#Pre Process Raw Data
copy = binarize_labels(copy)
copy = NA_fixer(copy)

In [6]:
#Split to X,y
X = copy.drop('RiskPerformance', axis = 1) #Create X Variable by dropping y

y = copy.RiskPerformance #Create y variable


In [7]:
#Get numeric attributes and categoric attributes

cat_cols = ['MaxDelq2PublicRecLast12M', 'MaxDelqEver']

int_df = X.drop(cat_cols, axis = 1)
num_cols = int_df.columns

In [8]:
#Split into Training and test data
from sklearn.model_selection import train_test_split




X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 1)

#IF WE NEED 3 DATAS, UNCOMMENT THIS and COMMENT OUT THE THE ABOVE SPLIT (X_valid will be the data we use in the interface)
# X_modeling, X_valid, y_modeling, y_valid = train_test_split(X,y, test_size = 0.01, random_state = 1)
# X_train, X_test, y_train, y_test = train_test_split(X_modeling,y_modeling, test_size = 0.2, random_state = 1)

In [9]:
#Create a Pipeline to Do data transformation (below cells) all at once
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer



def pipe(df, num_cols, cat_cols):
    
    num_pipeline = Pipeline([
            ('imputer', SimpleImputer(strategy="median")),
            ('std_scaler', StandardScaler()),
        ])
    cat_pipeline = Pipeline([
            ('imputer', SimpleImputer(strategy="most_frequent")),
            ('OH_enc', OneHotEncoder()),
        ])

    full_pipeline = ColumnTransformer([
            ("num", num_pipeline, num_cols),
            ("cat", cat_pipeline, cat_cols),
        ])

    return pd.DataFrame(full_pipeline.fit_transform(df), index = df.index)




In [10]:
#Transform Data with Pipeline
X_train_complete = pipe(X_train, num_cols, cat_cols)
X_test_complete = pipe(X_test, num_cols, cat_cols)

In [11]:
best_gboost = GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.15000000000000002, loss='deviance',
                           max_depth=3, max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=150,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)
best_SVC = SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
                decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
                max_iter=-1, probability=False, random_state=None, shrinking=True,
                tol=0.001, verbose=False)
best_Logit = LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

SyntaxError: invalid syntax (<ipython-input-11-22721a70cfb0>, line 11)

In [ ]:
#Model Pipelines
Boost_Pipeline = Pipeline([
        ('data_cleaner', pipe),
        ('model', best_gboost)
    ])
SVC_Pipeline = Pipeline([
        ('data_cleaner', pipe),
        ('model', best_SVC)
    ])
Logit_Pipeline = Pipeline([
        ('data_cleaner', pipe),
        ('model', best_Logit)
    ])

In [12]:
#Test Model

def class_model_test(model):
    model.fit(X_train_complete,y_train)
    y_pred = model.predict(X_test_complete)
    #mse = mean_squared_error(y_test, y_pred)
    #rmse = np.sqrt(mse)
    #return rmse
    prfs =  precision_recall_fscore_support(y_test, y_pred, average = 'micro')
    df = pd.DataFrame(list(precision_recall_fscore_support(y_test, y_pred, average = 'micro')), index = ['Precision', 'Recall', 'F Score', 'Support'])
    return df
    #return prfs



In [13]:
model_names = ['SVC', 'Logit', 'Decision Tree', 'Random Forest', 'MLP',
               'Decision Tree Classifier','KNN','Ada Boost', 'Gaussian NB', 'Quadratic Discrimanant Analysis',
               'Gradient Boosting', 'Bagging', 'Extra Trees']
models = [SVC(), LogisticRegression(), DecisionTreeClassifier(), RandomForestClassifier(), MLPClassifier(), DecisionTreeClassifier(), KNeighborsClassifier(), 
          AdaBoostClassifier(), GaussianNB(), QuadraticDiscriminantAnalysis(), GradientBoostingClassifier(),
          BaggingClassifier(), ExtraTreesClassifier()]


result_list = []
for i in range(len(models)):
    result = class_model_test(models[i])
    print('model ' + model_names[i] + ' tested')
    result_list.append([model_names[i], result[0][2]])
    model_test_results = pd.DataFrame(result_list, columns=['Model', 'F Score'])
#model_test_results

model SVC tested
model Logit tested
model Decision Tree tested
model Random Forest tested


C:\Users\isaac\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


model MLP tested
model Decision Tree Classifier tested
model KNN tested
model Ada Boost tested
model Gaussian NB tested
model Quadratic Discrimanant Analysis tested


C:\Users\isaac\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


model Gradient Boosting tested
model Bagging tested
model Extra Trees tested


,Model,F Score
0,SVC,0.715105
1,Logit,0.707935
2,Decision Tree,0.624283
3,Random Forest,0.689293
4,MLP,0.708891
5,Decision Tree Classifier,0.623327
6,KNN,0.676386
7,Ada Boost,0.710325
8,Gaussian NB,0.636711
9,Quadratic Discrimanant Analysis,0.618547


In [ ]:
#Hyper Tune Models That Perform the Best

from sklearn.model_selection import GridSearchCV

# SVC = SVC()
# params = {'C': [1,5], 'gamma': [0.5, 0.1, .001], 'kernel': ['rbf','poly']} #'degree': [2,3,4]}

# logit = LogisticRegression()
# params = {'C': [0.1,0.5,1,2]}

grid_search = GridSearchCV(SVC, params, cv = 5)
grid_search.fit(X_train_complete, y_train)
grid_search.best_estimator_

In [ ]:
best_model = grid_search.best_estimator_
best_model